# 一个简单例子
用到random seed和for循环和boolean表达式
生成随机X，Y是两特征的和，小于1看作True，1,

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
BATCH_SIZE = 8
DATA_SIZE = 32
seed = 23455

In [3]:
rng = np.random.RandomState(seed)
print(rng)
print(type(rng))
X = rng.rand(DATA_SIZE,2)
print(X[-5:])

<class 'mtrand.RandomState'>
[[ 0.06727785  0.57784761]
 [ 0.38492705  0.48384792]
 [ 0.69234428  0.19687348]
 [ 0.42783492  0.73416985]
 [ 0.09696069  0.04883936]]


In [4]:
#用表达式求Y的写法，不过这种写法也不算很好，毕竟很慢。但是这种判别式加赋值，能向量化吗？
Y = [[int(x0 + x1 < 1)] for (x0, x1) in X]
print(Y[-5:])

[[1], [1], [1], [0], [1]]


In [5]:
#定义模型
x = tf.placeholder(tf.float32, shape = (None, 2))
y_ = tf.placeholder(tf.float32, shape = (None, 1))
w1 = tf.Variable(tf.random_normal([2,3], stddev = 1, seed = 1))
w2 = tf.Variable(tf.random_normal([3,1], stddev = 1, seed = 1))
a = tf.matmul(x,w1)
y = tf.matmul(a,w2)

loss = tf.reduce_mean(tf.square(y-y_))
train_step = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

In [15]:
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print('before train:\nw1:',sess.run(w1))
    print('w2:',sess.run(w2))
    STEPS = 3000
    for i in range(STEPS):
        start = (i * BATCH_SIZE) % (DATA_SIZE)#数据集分成四batch。
        end = start + BATCH_SIZE
#         print('start:',start)
#         print(len(Y))
#         print(len(Y[0]))
        sess.run(train_step,{x:X[start:end], y_:Y[start:end]})
#         sess.run(train_step,{x:X[start:end], y:Y[start:end]})
        if i % 500 == 0:
            print('loss is :',sess.run(loss,{x:X,y_:Y}))
        

    print('after train:\nw1:',sess.run(w1))
    print('w2:',sess.run(w2))

before train:
w1: [[-0.81131822  1.48459876  0.06532937]
 [-2.44270396  0.0992484   0.59122431]]
w2: [[-0.81131822]
 [ 1.48459876]
 [ 0.06532937]]
loss is : 5.13118
loss is : 0.429111
loss is : 0.409789
loss is : 0.399923
loss is : 0.394146
loss is : 0.390597
after train:
w1: [[-0.70006633  0.9136318   0.08953569]
 [-2.34024906 -0.1464127   0.58823055]]
w2: [[-0.06024269]
 [ 0.91956186]
 [-0.06820712]]


## 关于placeholder报错

并没有定义过placeholder 'y'，但是如果直接使用y,不是提示没有y之类的，是说有负dimensions
所以感觉还是有些默认规则
原则上，这个字典应该是你自己随便传入的

        sess.run(train_step,{x:X[start:end], y:Y[start:end]})
        
    InvalidArgumentError (see above for traceback): Shape [-1,1] has negative dimensions
         [[Node: Placeholder_1 = Placeholder[dtype=DT_FLOAT, shape=[?,1], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]
     
     
但是：

sess.run(train_step,{x:X[start:end], y_:Y[start:end],y2:Y[start:end]})
y我也没定义，为什么y的错误就和y2不一样？
        
     NameError: name 'y2' is not defined
     
原因：y是有定义的，不是placeholder

    y = tf.matmul(a,w2)

不报错：

        sess.run(train_step,{x:X[start:end], y_:Y[start:end],y:Y[start:end]})
报错：

        sess.run(train_step,{x:X[start:end], y:Y[start:end]})
所以这个错误的本质是，你没把y_这个placeholder填充起来，和负不负的没关系。        

In [62]:
print(X[start:end,:])
print(len(X[start:end,:]))
print((X[start:end]))
print(Y[start:end])

[[ 0.83494319  0.11482951]
 [ 0.66899751  0.46594987]
 [ 0.60181666  0.58838408]
 [ 0.31836656  0.20502072]
 [ 0.87043944  0.02679395]
 [ 0.41539811  0.43938369]
 [ 0.68635684  0.24833404]
 [ 0.97315228  0.68541849]]
8
[[ 0.83494319  0.11482951]
 [ 0.66899751  0.46594987]
 [ 0.60181666  0.58838408]
 [ 0.31836656  0.20502072]
 [ 0.87043944  0.02679395]
 [ 0.41539811  0.43938369]
 [ 0.68635684  0.24833404]
 [ 0.97315228  0.68541849]]
[[1], [0], [0], [1], [1], [1], [1], [0]]


In [ ]:
#好像除了写for循环，没其他办法把这个表达式得到的boolean类型转成int了
#Y2 = int(X[:,0] + X[:,1] < 1)#不能直接给list转成int
Y2 = X[:,0] + X[:,1] < 1
#Y3 = list(Y2,dtype=int)
Y3 = [int(i) for i in Y2]
print(Y2)
print(Y3)

In [30]:
#随机数测试
ww = tf.random_normal([2,3],stddev=1,seed=1)
print(ww)
with tf.Session() as sess:
    print(sess.run(ww))

Tensor("random_normal_3:0", shape=(2, 3), dtype=float32)
[[-0.81131822  1.48459876  0.06532937]
 [-2.44270396  0.0992484   0.59122431]]
